In [2]:
import torch.nn as nn
import torch.nn.functional as F

1. 定义一个类继承于父类nn.Module
2. 两个函数
    1) \_\_init\_\_( self, in_channels(特征矩阵深度)， out_channels（卷积核个数）, kernel_size(卷积核大小）， stride = 1, padding = 0, ...
        Conv2d( 深度， 卷积核个数， 卷积核大小（边长） ）
        最大下采样：MaxPoolNd( 卷积核大小, stride=不定义即为卷积核大小, ...
        Linear 展平
   2) forward
        结合卷积结果和激活函数
        view函数：展平成一维

In [3]:
class LeNet(nn.Module):
    def __init__(self):
        super( LeNet, self ).__init__()      #解决继承过程中的问题
        self.conv1 = nn.Conv2d( 3, 16, 5 )
        self.pool1 = nn.MaxPool2d( 2, 2 )   # 改变高和宽，不影响深度 eg. 16, 28, 28 -> 16, 14, 14
        self.conv2 = nn.Conv2d( 16, 32, 5 ) # 第2个conv输出的卷积核深度为16，到第2个卷积核深度
        self.pool2 = nn.MaxPool2d( 2, 2 )
        self.fc1 = nn.Linear( 32*5*5, 120 )
        self.fc2 = nn.Linear( 120, 84 )
        self.fc3 = nn.Linear( 84, 10 )
    def forward(self, x):
        x = F.relu( self.conv1( x ) )       # in: 3, 32, 32  out: 16, 28, 28
        x = self.pool1( x )                 # out: 16, 14, 14
        x = F.relu( self.conv2( x ) )       # out: 32, 10, 10
        x = self.pool2( x )                 # out: 32, 5, 5
        x = x.view( -1, 32*5*5 )            # out: 32*5*5
        x = F.relu( self.fc1( x ) )         # out: 120
        x = F.relu( self.fc2( x ) )         # out: 84
        x = self.fc3( x )                   # out: 10
        return x

测试

In [4]:
'''
import torch
in1 = torch.rand( [32, 3, 32, 32] )
model = LeNet()
print( model )
out1 = model( in1 )
'''

'\nimport torch\nin1 = torch.rand( [32, 3, 32, 32] )\nmodel = LeNet()\nprint( model )\nout1 = model( in1 )\n'

**训练集**
transform -> 预处理
ToTensor：Numpy/Img -> tensor
          高度\*宽度\*深度 -> 0\*0\*1
datasets：自带数据集，root-存储位置

In [5]:
import torch
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

transform = transforms.Compose( [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10( root = './data', train = True, download = True, transform = transform )
trainloader = DataLoader( trainset, batch_size = 64, shuffle = True, num_workers =0 )

Files already downloaded and verified


测试集

In [6]:
testset = torchvision.datasets.CIFAR10( root = './data', train = False, download = False, transform = transform )
testloader = DataLoader( testset, batch_size = 5000, shuffle = False, num_workers =0 )

test_data_iter = iter( testloader )
test_images, test_labels = next( test_data_iter )

classes = ( 'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck' )

In [7]:
'''
def imshow(img):
    img = img / 2 * 0.5                                             # 还原transform对数据的预处理
    npimg = img.numpy()                                             # 转化成numpy格式
    plt.imshow( np.transpose( npimg, ( 1, 2, 0 ) ) )                # 高度，宽度，深度
    plt.show
    
    
print( ' '.join( '%s' % classes[ test_labels[j] ] for j in range( 4 ) ) )
imshow( torchvision.utils.make_grid( test_images ) )
'''

"\ndef imshow(img):\n    img = img / 2 * 0.5                                             # 还原transform对数据的预处理\n    npimg = img.numpy()                                             # 转化成numpy格式\n    plt.imshow( np.transpose( npimg, ( 1, 2, 0 ) ) )                # 高度，宽度，深度\n    plt.show\n    \n    \nprint( ' '.join( '%s' % classes[ test_labels[j] ] for j in range( 4 ) ) )\nimshow( torchvision.utils.make_grid( test_images ) )\n"

In [8]:
net = LeNet()
loss_f = nn.CrossEntropyLoss()                                      # 损失函数
optimizer = optim.Adam( net.parameters(), lr = 0.001 )              # lr -> learning rate

# 迭代训练集
for ep in range( 5 ):
    running_loss = 0.0
    for step, data in enumerate( trainloader, start = 0 ):
        inputs, labels = data
        optimizer.zero_grad()                                       # 清空历史梯度
        outputs = net( inputs )
        loss = loss_f( outputs, labels )
        loss.backward()                                             # 反向传播
        optimizer.step()
        
        running_loss += loss.item()
        if step % 500 == 499:                                       # 每500步打印
            with torch.no_grad():                                   # 测试过程中不计算每个节点的误差损失梯度
                outputs = net( test_images )
                predict_y = torch.max( outputs.data, dim = 1 )[1]   # 记录index
                accuracy = torch.eq( predict_y, test_labels ).sum().item() / test_labels.size(0)
                print( '[%d, %5d] loss:%.3f accuracy:%.3f' % ( ep + 1, step + 1, running_loss / 500, accuracy ) )
                running_loss = 0.0

[1,   500] loss:1.658 accuracy:0.475
[2,   500] loss:1.260 accuracy:0.558
[3,   500] loss:1.076 accuracy:0.632
[4,   500] loss:0.970 accuracy:0.649
[5,   500] loss:0.882 accuracy:0.670


In [9]:
# 保存模型
save_path = './path/Lenet.pth'
torch.save(net.state_dict(), save_path)
print( 'finished')

finished


预测

In [10]:
from PIL import Image

transform = transforms.Compose([transforms.Resize( ( 32, 32 ) ), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

net.load_state_dict(torch.load('./path/Lenet.pth'))

im = Image.open('./data/test2.jpg')
im = transform(im)                                      # 高度，宽度，深度
im = torch.unsqueeze( im, dim = 0 )                     # 增加新的维度：batch，高度，宽度，深度

with torch.no_grad():
    outputs = net( im )
    predict = torch.max( outputs, dim = 1 )[1].numpy()  # 只取index
print( classes[int(predict)] )

cat


C:\Users\EC319\AppData\Local\Temp\ipykernel_15660\161287143.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print( classes[int(predict)] )
